In [86]:
import pandas as pd
import monthdelta

In [87]:
cpi = pd.read_csv('data/cpi-ocde.csv')

In [88]:
cpi.columns = cpi.columns.str.lower()

In [89]:
cpi.head()

,location,indicator,subject,measure,frequency,time,value,flag codes
0,AUS,CPI,ENRG,AGRWTH,A,1972,4.910070,NaN
1,AUS,CPI,ENRG,AGRWTH,A,1973,3.762801,NaN
2,AUS,CPI,ENRG,AGRWTH,A,1974,13.173540,NaN
3,AUS,CPI,ENRG,AGRWTH,A,1975,19.422470,NaN
4,AUS,CPI,ENRG,AGRWTH,A,1976,8.833195,NaN


In [90]:
cpi['subject'].unique()

array(['ENRG', 'FOOD', 'TOT', 'TOT_FOODENRG'], dtype=object)

In [91]:
cpi_int = cpi.query('(frequency=="M") & (subject=="TOT") & (measure=="AGRWTH")')

In [92]:
cpi_int = cpi_int[['location', 'time', 'value']]

In [93]:
cpi_int.sort_values(['location', 'time'], inplace=True)

In [94]:
# WE remove ARG
cpi_int = cpi_int[~(cpi_int['location'] == 'ARG')]

In [95]:
cpi_int.head()

,location,time,value
30128,AUT,1959-01,-0.401606
30129,AUT,1959-02,-0.501002
30130,AUT,1959-03,-0.401204
30131,AUT,1959-04,-1.283317
30132,AUT,1959-05,-1.851852


In [96]:
#Only data before 2020
cpi_int = cpi_int[cpi_int['time']<'2020-01']

In [97]:
dates_val_test = cpi_int.groupby(['location']).agg({'time': 'max'}).reset_index()

In [98]:
dates_val_test['max_test_date'] = (pd.to_datetime(dates_val_test['time'], format='%Y-%m').dt.date - monthdelta.monthdelta(3*12))
dates_val_test['max_test_date'] = pd.to_datetime(dates_val_test['max_test_date']).dt.strftime('%Y-%m')

In [99]:
dates_val_test.drop(columns='time', inplace=True)

In [100]:
cpi_int = cpi_int.merge(dates_val_test, how='left', on=['location'])

In [101]:
cpi_int.head()

,location,time,value,max_test_date
0,AUT,1959-01,-0.401606,2016-12
1,AUT,1959-02,-0.501002,2016-12
2,AUT,1959-03,-0.401204,2016-12
3,AUT,1959-04,-1.283317,2016-12
4,AUT,1959-05,-1.851852,2016-12


In [102]:
cpi_int.loc[cpi_int['time'] < cpi_int['max_test_date'], 'type'] = 'train'
cpi_int.loc[cpi_int['time'] >= cpi_int['max_test_date'], 'type'] = 'test'

In [103]:
cpi_int = cpi_int[['location', 'time', 'value', 'type']]

In [104]:
cpi_int.to_csv('data/data_cpi/cpi-ocde-val.csv', index=False)